In [1]:
import duckdb
from pathlib import Path
import datetime

In [2]:
ts = datetime.datetime.now()

In [3]:
base_path = Path().cwd().parent
source_path = base_path / Path('raw/entity/anime')
target_path = base_path / Path(f'silver/anilist/link/link-{ts.year}-{ts.month}-{ts.day}.parquet')

In [4]:
tb_links = duckdb.sql(f"""
    WITH source AS(
    SELECT * FROM read_json_auto('{str(source_path)}/*.json')
    )
    , extracted_to_list AS (
    SELECT (source->>'Page'->>'media')::JSON[] as anime_list FROM source
    )
    , unnest_ AS (
    SELECT unnest(anime_list) as anime FROM extracted_to_list
    )
    , extracted_attributes_list AS (
        SELECT
        json_extract_string(anime,
        [
            'id'
            , '$.externalLinks'

        ]) attribute_list
        FROM unnest_
    )
    , tb_links AS (
        SELECT
        attribute_list[1]::INT AS id
        , json_extract_string(unnest(attribute_list[2]::JSON[]),
        [
            '$.id'
            , '$.url'
            , '$.site'
            , '$.type'
            , '$.siteId'
            , '$.language'
        ]) attribute_list_link
        FROM extracted_attributes_list
    )
    SELECT
        attribute_list_link[1]::INT AS id
        , attribute_list_link[2]::VARCHAR AS url
        , attribute_list_link[3]::VARCHAR AS site
        , attribute_list_link[4]::VARCHAR AS type
        , attribute_list_link[5]::INT AS siteId
        , attribute_list_link[6]::VARCHAR AS language
        , id AS anime_id
    FROM tb_links
""")

In [8]:
tb_links.to_parquet(str(target_path))